## Table of Contents

**Part 1: Data Cleaning, EDA and Feature Engineering** 
- Train Data Cleaning, EDA and Feature Engineering
- Weather Data Cleaning, EDA and Feature Engineering
- Spray Data Cleaning, EDA and Feature Engineering
- Combined EDA and Feature Engineering

**Part 2: Preprocessing, Feature Selection and Baseline Model** 
- Pre-processing
- Feature Selection
- Baseline Model

**Part 3: Modelling**
- Model Fit and Testing
- Model Iteration
- Model Evaluation
- Conclusion & Recommending

## Module Imports

In [25]:
# Standard imports
import numpy as np # Version 1.20.1
import pandas as pd # Version 1.2.4
# Pandas settings
pd.set_option('display.max_rows', None)
pd.set_option("display.max_columns", None)

# Plotting imports
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('ticks')
# Please reduce "DPI" values below if plots take too long to display 
# Higher values make plots remain clear when zoomed in
plt.rcParams['figure.dpi'] = 200
# The lowest recommended value is 200 for A4 size print legibility 
# The recommended value is 600 for 'photograph-like' legibility
plt.style.use('ggplot')
%matplotlib inline

# Time Series imports
import datetime as dt # Version 2.8.1

# Pre-Processing imports
import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline as pipe
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

# Modelling imports
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, plot_roc_curve, precision_recall_fscore_support,  roc_auc_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

## Function definition

## Pre-precessing

In [2]:
train_combined = pd.read_csv('../data/train_combined.csv')
test = pd.read_csv('../data/test.csv', parse_dates = ['Date'])
weather_combined = pd.read_csv('../data/weather_combined.csv', parse_dates = ['Date'])

In [3]:
train_combined.drop(columns = 'Unnamed: 0', inplace = True)

In [4]:
test.shape

(116293, 11)

In [5]:
test_merged = test.merge(weather_combined[weather_combined['Station']==1], on='Date')

In [6]:
test_merged.columns

Index(['Id', 'Date', 'Address', 'Species', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy',
       'Unnamed: 0', 'index', 'Station', 'Tmax', 'Tmin', 'Tavg', 'DewPoint',
       'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset', 'Wet_NoWet',
       'PrecipTotal', 'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir',
       'AvgSpeed', 'Year', 'Month', 'WeekofYear', 'Daylight_Hours', 'Tmin_rm7',
       'DewPoint_rm7', 'Sunrise_rm7', 'PrecipTotal_rm7', 'StnPressure_rm7',
       'ResultSpeed_rm7', 'ResultDir_rm7', 'Tmin_rm7_lag1', 'Tmin_rm7_lag2',
       'Tmin_rm7_lag3', 'Tmin_rm7_lag4', 'DewPoint_rm7_lag1',
       'DewPoint_rm7_lag2', 'DewPoint_rm7_lag3', 'DewPoint_rm7_lag4',
       'Sunrise_rm7_lag1', 'Sunrise_rm7_lag2', 'Sunrise_rm7_lag3',
       'Sunrise_rm7_lag4', 'PrecipTotal_rm7_lag1', 'PrecipTotal_rm7_lag2',
       'PrecipTotal_rm7_lag3', 'PrecipTotal_rm7_lag4', 'StnPressure_rm7_lag1',
       'StnPressure_rm7_lag2', 'StnPressure_r

In [7]:
test_merged.drop(columns = ['Id','Address','Block','Street','AddressNumberAndStreet','AddressAccuracy','Unnamed: 0'], inplace = True)
# Dropping identified collinear weather features
test_merged.drop(columns = ['Tmax','Tavg','Heat','Cool'], inplace = True)
test_merged.drop(columns = 'WetBulb', inplace = True)
test_merged.drop(columns = ['Sunset','Daylight_Hours'], inplace = True)
test_merged.drop(columns = ['SeaLevel','AvgSpeed'], inplace = True)

# Drop Trap column as traps are identifiable through Latitude and Longitude values
test_merged.drop(columns = 'Trap', inplace = True)
# Drop the Station there is only one unique value(Station 1)
test_merged.drop(columns = 'Station', inplace = True)
# Drop roll mean columns as train dataset interval is weekly
test_merged.drop(columns = ['Tmin_rm7','DewPoint_rm7','Sunrise_rm7','PrecipTotal_rm7','StnPressure_rm7','ResultSpeed_rm7','ResultDir_rm7'], inplace = True)
# Drop index column from merge
test_merged.drop(columns = 'index', inplace = True)

# Create new date & time related columns; i.e. month and weekofyear
test_merged['Month'] = test_merged['Date'].dt.month
test_merged['WeekofYear'] = test_merged['Date'].dt.isocalendar().week
# Drop date as we no longer need with the new dt columns
test_merged.drop(columns = 'Date', inplace = True)
# Encode Species column to indicate 1 for species that carry WNV and 0 for species that do not 
test_merged['Species'] = test_merged['Species'].apply(lambda x: 1 if x in ['CULEX PIPIENS','CULEX PIPIENS/RESTUANS','CULEX RESTUANS'] else 0)

In [8]:
train_combined.shape

(8610, 43)

In [9]:
test_merged.shape

(116293, 42)

In [10]:
test_merged['Year'].unique()

array([2008, 2010, 2012, 2014], dtype=int64)

### Spliting train and test

Here is an outline of how the train test datasets will be transformed:
- Train data to be split into four sets consisting of the following years:
    - train1 (2007)
    - train2 (2007,2009)
    - train3 (2007,2009,2011)
    - train4 (2009,2011,2013)  
      
      
- Test data to be split into four sets consisting of the following years:
    - test1 (2008)
    - test2 (2010)
    - test3 (2012)
    - test4 (2014)  
    
In practice, we very likely will retrain our model as new data becomes available.

This would give the model the best opportunity to make good forecasts at each time step. We can evaluate our machine learning models under this assumption.

Starting at the beginning of the time series, the minimum number of samples in the window is used to train a model.
1. The model makes a prediction for the next time step.
2. The prediction is stored or evaluated against the known value.
3. The window is expanded to include the known value and the process is repeated (go to step 1.)  

Because this methodology involves moving along the time series one-time step at a time, it is often called Walk Forward Testing or Walk Forward Validation. Additionally, because a sliding or expanding window is used to train a model, this method is also referred to as Rolling Window Analysis or a Rolling Forecast.  

It should also be important to note the above methodology cannot be applied to this project as ideally once a year has passed in the present we would have obtained the ground truth for that year for both comparing model performance and updating the train sets to include years passed (we did not include test years passed as the training data for subsequent years).

The rationale for the split is in pursuit of realism, for time-series predictions there are assumptions made where past data are correlated to the present. Not only is past data correlated, another assumption is that the most recent past data have a natural tendency to be the most correlated.

Secondly, it would not be realistically correct to have future data(which is what we are trying to predict) available to train models, which would be the case if we were to use models trained on the entire train dataset (every alt. year from 2007 - 2013) to predict for year 2008.

Thirdly, we decided to limit the maximum number of years in a training set to 3 years, this will allow us to keep only the three most recent years and also prevent an ever expanding training set which will become more computationally intensive in both resource and time to maintain and train.




In [11]:
# Splitting of train dataset into the outlined years
tr01 = train_combined[train_combined['Year'] == 2007]
tr02 = train_combined[(train_combined['Year'] == 2007) | (train_combined['Year'] == 2009)]
tr03 = train_combined[(train_combined['Year'] == 2007) | (train_combined['Year'] == 2009) | (train_combined['Year'] == 2011)]
tr04 = train_combined[(train_combined['Year'] == 2009) | (train_combined['Year'] == 2011) | (train_combined['Year'] == 2013)]

In [12]:
# Splitting of test dataset into the outlined years
tst01 = test_merged[test_merged['Year'] == 2008]
tst02 = test_merged[test_merged['Year'] == 2010]
tst03 = test_merged[test_merged['Year'] == 2012]
tst04 = test_merged[test_merged['Year'] == 2014]

### Baseline Model

As we split our train dataset into 4, the baseline models for each respective sets will be their respective normalized value counts.

In [13]:
print("trainset1")
print(tr01.shape)
print(tr01['WnvPresent'].value_counts(normalize = True))

trainset1
(2837, 43)
0    0.93338
1    0.06662
Name: WnvPresent, dtype: float64


In [14]:
print("trainset2")
print(tr02.shape)
print(tr02['WnvPresent'].value_counts(normalize = True))

trainset2
(4758, 43)
0    0.956284
1    0.043716
Name: WnvPresent, dtype: float64


In [15]:
print("trainset3")
print(tr03.shape)
print(tr03['WnvPresent'].value_counts(normalize = True))

trainset3
(6552, 43)
0    0.960623
1    0.039377
Name: WnvPresent, dtype: float64


In [16]:
print("trainset4")
print(tr04.shape)
print(tr04['WnvPresent'].value_counts(normalize = True))

trainset4
(5773, 43)
0    0.953577
1    0.046423
Name: WnvPresent, dtype: float64


While we have fluctuating percentage values for our negative class (WnvPresent:0), imbalanced classes is common across all training sets, we will deal with it using `SMOTE`.


## Metrics for model evaluation

While the `AUC-ROC` score will be evaluated for Kaggle, the more important metric to focus on for the problem statement is false negatives. 

False negatives happen when the model wrongly predicts no presence of WNV when it is actually present. It will be important to ensure minimal false negatives these may translate to outbreaks causing enormous strain on public healthcare and ineffective allocation of resources. The model metric allowing us to measure model performance with regards to minimizing false negatives is `Recall` score.

Ultimately, when evaluating model performance we will prioritize choosing models with primarily high recall score, followed by AUC-ROC score.

In [17]:
ss = StandardScaler()
smt = SMOTE(random_state = 42)

In [18]:
pipe = 

In [19]:
# Define X and Y for tr01
X = tr01.drop(columns=['WnvPresent'])
y = tr01['WnvPresent']

# Train,test split using date based slicing
train_size = int(len(X) * 0.80)
X_tr01, X_val01 = X[0:train_size], X[train_size:len(X)]
y_tr01, y_val01 = y[0:train_size], y[train_size:len(X)]
print('Observations: %d' % (len(X)))
print('Training Observations: %d' % (len(X_tr01)))
print('Validation Observations: %d' % (len(X_val01)))

# Scaling X_train , X_val
X_tr01sc = ss.fit_transform(X_tr01)
X_val01sc = ss.transform(X_val01)

# Oversampling using SMOTE
X_tr01sc_smt, y_tr01_smt = smt.fit_resample(X_tr01sc, y_tr01)
print('X_tr01 scaled sampled  observations: %d' % (len(X_tr01sc_smt)))
print('y_tr01 sampled observations: %d' % (len(y_tr01_smt)))

# trainset1 modelling final variables
# X_train, y_train : X_tr01sc_smt, y_tr01_smt
# X_val, y_val : X_val01sc, y_val01

Observations: 2837
Training Observations: 2269
Validation Observations: 568
X_tr01 scaled sampled  observations: 4168
y_tr01 sampled observations: 4168


In [20]:
# Define X and Y for tr02
X = tr02.drop(columns=['WnvPresent'])
y = tr02['WnvPresent']

# Train,test split using date based slicing
train_size = int(len(X) * 0.80)
X_tr02, X_val02 = X[0:train_size], X[train_size:len(X)]
y_tr02, y_val02 = y[0:train_size], y[train_size:len(X)]
print('Observations: %d' % (len(X)))
print('Training Observations: %d' % (len(X_tr02)))
print('Validation Observations: %d' % (len(X_val02)))

# Scaling X_train , X_val
X_tr02sc = ss.fit_transform(X_tr02)
X_val02sc = ss.transform(X_val02)

# Oversampling using SMOTE
X_tr02sc_smt, y_tr02_smt = smt.fit_resample(X_tr02sc, y_tr02)
print('X_tr02 scaled sampled  observations: %d' % (len(X_tr02sc_smt)))
print('y_tr02 sampled observations: %d' % (len(y_tr02_smt)))

# trainset1 modelling final variables
# X_train, y_train : X_tr02sc_smt, y_tr02_smt
# X_val, y_val : X_val02sc, y_val02

Observations: 4758
Training Observations: 3806
Validation Observations: 952
X_tr02 scaled sampled  observations: 7230
y_tr02 sampled observations: 7230


In [21]:
# Define X and Y for tr03
X = tr03.drop(columns=['WnvPresent'])
y = tr03['WnvPresent']

# Train,test split using date based slicing
train_size = int(len(X) * 0.80)
X_tr03, X_val03 = X[0:train_size], X[train_size:len(X)]
y_tr03, y_val03 = y[0:train_size], y[train_size:len(X)]
print('Observations: %d' % (len(X)))
print('Training Observations: %d' % (len(X_tr03)))
print('Validation Observations: %d' % (len(X_val03)))

# Scaling X_train , X_val
X_tr03sc = ss.fit_transform(X_tr03)
X_val03sc = ss.transform(X_val03)

# Oversampling using SMOTE
X_tr03sc_smt, y_tr03_smt = smt.fit_resample(X_tr03sc, y_tr03)
print('X_tr03 scaled sampled  observations: %d' % (len(X_tr03sc_smt)))
print('y_tr03 sampled observations: %d' % (len(y_tr03_smt)))

# trainset1 modelling final variables
# X_train, y_train : X_tr03sc_smt, y_tr03_smt
# X_val, y_val : X_val03sc, y_val03

Observations: 6552
Training Observations: 5241
Validation Observations: 1311
X_tr03 scaled sampled  observations: 10066
y_tr03 sampled observations: 10066


In [22]:
# Define X and Y for tr04
X = tr04.drop(columns=['WnvPresent'])
y = tr04['WnvPresent']

# Train,test split using date based slicing
train_size = int(len(X) * 0.80)
X_tr04, X_val04 = X[0:train_size], X[train_size:len(X)]
y_tr04, y_val04 = y[0:train_size], y[train_size:len(X)]
print('Observations: %d' % (len(X)))
print('Training Observations: %d' % (len(X_tr04)))
print('Validation Observations: %d' % (len(X_val04)))

# Scaling X_train , X_val
X_tr04sc = ss.fit_transform(X_tr04)
X_val04sc = ss.transform(X_val04)

# Oversampling using SMOTE
X_tr04sc_smt, y_tr04_smt = smt.fit_resample(X_tr04sc, y_tr04)
print('X_tr04 scaled sampled  observations: %d' % (len(X_tr04sc_smt)))
print('y_tr04 sampled observations: %d' % (len(y_tr04_smt)))

# trainset1 modelling final variables
# X_train, y_train : X_tr04sc_smt, y_tr04_smt
# X_val, y_val : X_val04sc, y_val04

Observations: 5773
Training Observations: 4618
Validation Observations: 1155
X_tr04 scaled sampled  observations: 9046
y_tr04 sampled observations: 9046
